# 自定义层

我们推荐使用`tf.keras`作为一个高层API来构建神经网络。大多数TensorFlow API可以基于即刻执行使用。 


In [ ]:
import tensorflow as tf

In [ ]:
print(tf.test.is_gpu_available())

## 层：有用操作的一般集合

多数时候，你写机器学习模型代码时，你希望使用高层抽象，而不是单个操作和单个变量的操作。

很多机器学习模型通过组合和堆砌相对简单的层来表达，TensorFlow提供了很多通用层和简单方法让你编写具体应用层，可以从头编写，也可以组合现有层。

在tf.keras包中TensorFlow包括完整的[Keras](https://keras.io) API，在构建自己模型时，Keras 层非常有用。


In [ ]:
# 在tf.keras.layers包中，层是对象。通过构建对象创建层。 
# 多数层的第一个参数是输出维或信道的大小
layer = tf.keras.layers.Dense(100)

# 输入维的大小通常是不需要的，因为它可以在层第一次使用时推导出来。
# 但是，如果你想指定，可以给出，这在某些复杂模型中有用。
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

可以在文档中找到预先定义的层的完整列表。它包括Dense（完全连接的层），Conv2D，LSTM，BatchNormalization，Dropout等。

In [ ]:
# 通过调用使用层.
layer(tf.zeros([10, 5]))

In [ ]:
# 层有很多有用的属性和方法。例如，你可以使用`layer.variables` 和
# `layer.trainable_variables`查看层的变量.
# 在这种情况下，全连接层有权值和偏移变量.
layer.variables

In [ ]:
# 变量也可以通过访问器来访问T
layer.kernel, layer.bias

## 实现定制层

实现自己的层的最佳方法是扩展tf.keras.Layer类并实现：

1. `__init__ `，您可以在其中进行所有与输入无关的初始化
2. `build` ，您知道输入张量的形状，并可以进行其余的初始化
3. `call` ，在其中进行前向计算

请注意，您不必等到调用build来创建变量时，也可以在__init__创建它们。但是，在build中创建它们的优点是，它可以根据将在其上进行操作的输入的形状来进行后面变量创建。另一方面，在__init__创建变量将意味着需要明确指定创建变量所需的形状。

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [ ]:
_ = layer(tf.zeros([10, 5])) # 调用层

In [ ]:
print([var.name for var in layer.trainable_variables])

如果代码尽可能使用标准层，则更易于阅读和维护，因为其他读者会熟悉标准层的行为。

## 模型: 组合层


机器学习模型中许多有趣的类似于层的对象(模型)都是通过组合现有层来实现的。例如，resnet中的每个残差块都是卷积、批处理规范化和跳转连接的组合。层可以嵌套在其他层中。

通常，当您需要使用诸如`Model.fit` ， `Model.evaluate`和`Model.save`等模型方法时，您需要从`keras.Model`继承。

`keras.Model` （而不是keras.layers.Layer ）提供的另一个功能是，除了跟踪变量之外， `keras.Model`还跟踪其内部层，使它们更易于检查。

例如，这是一个ResNet块：

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [ ]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [ ]:
block.layers

In [ ]:
len(block.variables)

In [ ]:
block.summary()

但是，在很多时候，组成多层的模型只是简单地逐层调用。使用`tf.keras.Sequential`只需很少的代码即可完成：

In [ ]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

In [ ]:
my_seq.summary()